# OpenFermion Fermionic Quantum Emulator 
## Phase I Design Proposal

This document outlines the functionality of the API for the OpenFermion Fermionic Quantum Emulator as defined in the design proposal.

 The code is covered and tested with pytest-cov at 99%. Toggle the print option to see output.

In [32]:
fqeprint = True

### 2.A.2 API for data conversion between OpenFermion's backend and OpenFermion-FQE
The API supports data conversion between native Cirq wavefunctions and the FQE wavefunction for arbitrary particle number and spin configuration.  A linequbit wavefunction result is a normalized numpy array.

In [33]:
import fqe
import numpy
import cirq

nqubit = 4

cirq_wfn = numpy.random.rand(2**nqubit).astype(numpy.complex128) + numpy.random.rand(2**nqubit).astype(numpy.complex128)*1.j
norm = numpy.sqrt(numpy.vdot(cirq_wfn, cirq_wfn))
numpy.divide(cirq_wfn, norm, out=cirq_wfn)
wfn_from_cirq = fqe.from_cirq(cirq_wfn, 0.0001)

if fqeprint:
    print('Inital wavefunction is \n {} \n with norm {}'.format(cirq_wfn, numpy.vdot(cirq_wfn, cirq_wfn)))
    wfn_from_cirq.print_wfn()

Inital wavefunction is 
 [0.21663931+0.27930424j 0.27257157+0.1454505j  0.04599878+0.00051126j
 0.08000916+0.17675216j 0.13473104+0.23491062j 0.19019813+0.25923595j
 0.01266635+0.02180418j 0.27564601+0.00104139j 0.22790804+0.20073231j
 0.04860453+0.19916771j 0.24182489+0.12187263j 0.20211573+0.04055808j
 0.08880646+0.20968318j 0.04932386+0.24131864j 0.0975933 +0.19539965j
 0.22293679+0.16276882j] 
 with norm (1.0000000000000002+0j)
Sector N = 0 : S_z = 0
a'00'b'00' (0.21663930597277914+0.27930423733702975j)
Sector N = 1 : S_z = -1
a'00'b'01' (0.13473103833541647+0.2349106237566947j)
a'00'b'10' (0.272571569152339+0.14545050334405574j)
Sector N = 1 : S_z = 1
a'01'b'00' (0.22790803626781472+0.20073230740388734j)
a'10'b'00' (0.04599877510489929+0.0005112591366738259j)
Sector N = 2 : S_z = -2
a'00'b'11' (0.19019812996665186+0.25923595176167014j)
Sector N = 2 : S_z = 0
a'01'b'01' (0.08880646075217763+0.20968318030785582j)
a'01'b'10' (0.04860452901161002+0.19916770591494895j)
a'10'b'01' (-0.0

### 2.A.3 Initialization of wave functions
FQE wave functions can be initialized by calling the constructor directly. fqe.get_wavefunction and get_wavefunction_mutltiple are utility wrappers that provide wrappers for common initialization motifs. e.g. get_spin_nonconserving_wavefunction will build a wavefunction that permits spin breaking Operators to be applied while get_wavefunction_multiple will return a list of initialize wavefunctions

In [34]:
wfFqe = fqe.Wavefunction([[4, -2, 10]], broken=None)
wfFqe = fqe.get_wavefunction(4, -2, 10)

In [35]:
n = 3
m = -2
wfFQE = fqe.Wavefunction([[n, m, 10], [n+1, m+2, 10]], broken=None)
wfFQE = fqe.get_wavefunction_multiple([[4, 0, 10], [5, -5, 10]])

In [36]:
wfFQE = fqe.get_number_conserving_wavefunction(2, 4)
wfFQE.set_wfn(strategy='random')
if fqeprint:
    wfFQE.print_wfn()

Sector N = 2 : S_z = -2
a'0000'b'0011' (0.13741376984264198+0.21858138930916632j)
a'0000'b'0101' (0.17812903120313933+0.23050144667372388j)
a'0000'b'1001' (0.018493928708758628-0.803290479161759j)
a'0000'b'0110' (-0.01790010542315535-0.10757020379541009j)
a'0000'b'1010' (0.1077834079098812+0.3317034329962844j)
a'0000'b'1100' (-0.04538753591859354-0.25935482008351124j)
Sector N = 2 : S_z = 0
a'0001'b'0001' (-0.032160217105011144-0.10925842023669652j)
a'0001'b'0010' (-0.09759469091924397-0.08998677749039163j)
a'0001'b'0100' (0.22406346618992173+0.12802183687064878j)
a'0001'b'1000' (0.015177016285577489+0.07794949326641695j)
a'0010'b'0001' (0.1233684413471275-0.16701120663809563j)
a'0010'b'0010' (0.08465086261327827-0.09816117743994539j)
a'0010'b'0100' (0.20059792781307756-0.22999939274175885j)
a'0010'b'1000' (0.07310730363588082-0.12480494955030182j)
a'0100'b'0001' (0.06651284373056578-0.4142631154525293j)
a'0100'b'0010' (0.31429653318969253-0.1030206703270467j)
a'0100'b'0100' (0.1628510

### 2.A.4 Element access

Element access is modified from the original propsal to add more convient getter and setter functionalities.  The orginally proposed methods are availible with an addition function to add a value to an existing element.

In [37]:
wfFQE[(1, 8)] = .14159265359j
wfFQE[(1, 8)] += 3.j
imaginary_pi = wfFQE[(1, 8)]
if fqeprint:
    print(-imaginary_pi*1.j)

(3.14159265359-0j)


The new components not originally in the proposal allow access of entire vectors for getting and setting methods.

In [38]:
interesting_states = wfn_from_cirq.get_coeff((2, 0))
if fqeprint:
    print(interesting_states)

[[ 0.08880646+0.20968318j  0.04860453+0.19916771j]
 [-0.01266635-0.02180418j  0.08000916+0.17675216j]]


In [39]:
interesting_states[:, 1] += 1. + .1j
interesting_states *= .7 - .3j
if fqeprint:
    print(interesting_states)

[[ 0.12506948+0.12013629j  0.82377348-0.10516396j]
 [-0.0154077 -0.01146302j  0.83903206-0.13027624j]]


In [40]:
data = {}
data[(2, 0)] = interesting_states
wfn_from_cirq.set_wfn(strategy='from_data', raw_data=data)
if fqeprint:
    wfn_from_cirq.print_wfn()

Sector N = 0 : S_z = 0
a'00'b'00' (0.21663930597277914+0.27930423733702975j)
Sector N = 1 : S_z = -1
a'00'b'01' (0.13473103833541647+0.2349106237566947j)
a'00'b'10' (0.272571569152339+0.14545050334405574j)
Sector N = 1 : S_z = 1
a'01'b'00' (0.22790803626781472+0.20073230740388734j)
a'10'b'00' (0.04599877510489929+0.0005112591366738259j)
Sector N = 2 : S_z = -2
a'00'b'11' (0.19019812996665186+0.25923595176167014j)
Sector N = 2 : S_z = 0
a'01'b'01' (0.12506947661888107+0.12013628798984578j)
a'01'b'10' (0.8237734820826116-0.10516396456301871j)
a'10'b'01' (-0.015407701006962454-0.011463023308647397j)
a'10'b'10' (0.8390320620947339-0.13027623617672712j)
Sector N = 2 : S_z = 2
a'11'b'00' (0.24182488715712022+0.1218726307329772j)
Sector N = 3 : S_z = -1
a'01'b'11' (0.04932385684061608+0.24131863988711186j)
a'10'b'11' (-0.27564601461862587-0.0010413895352712407j)
Sector N = 3 : S_z = 1
a'11'b'01' (-0.09759329693001878-0.195399651060765j)
a'11'b'10' (0.2021157252054169+0.04055808074577214j)
Sec

### 2.A.5 Printing/Saving

Printing is currently availible as alpha beta strings followed by the coefficient as well as orbital occupation representation.

In [41]:
print('String forrmatting')
wfn_from_cirq.print_wfn(fmt='str')
print('\n Occupation forrmatting')
wfn_from_cirq.print_wfn(fmt='occ')

String forrmatting
Sector N = 0 : S_z = 0
a'00'b'00' (0.21663930597277914+0.27930423733702975j)
Sector N = 1 : S_z = -1
a'00'b'01' (0.13473103833541647+0.2349106237566947j)
a'00'b'10' (0.272571569152339+0.14545050334405574j)
Sector N = 1 : S_z = 1
a'01'b'00' (0.22790803626781472+0.20073230740388734j)
a'10'b'00' (0.04599877510489929+0.0005112591366738259j)
Sector N = 2 : S_z = -2
a'00'b'11' (0.19019812996665186+0.25923595176167014j)
Sector N = 2 : S_z = 0
a'01'b'01' (0.12506947661888107+0.12013628798984578j)
a'01'b'10' (0.8237734820826116-0.10516396456301871j)
a'10'b'01' (-0.015407701006962454-0.011463023308647397j)
a'10'b'10' (0.8390320620947339-0.13027623617672712j)
Sector N = 2 : S_z = 2
a'11'b'00' (0.24182488715712022+0.1218726307329772j)
Sector N = 3 : S_z = -1
a'01'b'11' (0.04932385684061608+0.24131863988711186j)
a'10'b'11' (-0.27564601461862587-0.0010413895352712407j)
Sector N = 3 : S_z = 1
a'11'b'01' (-0.09759329693001878-0.195399651060765j)
a'11'b'10' (0.2021157252054169+0.0405

### 2.A.6 Fermionic algebra operations and their unitaries on the state

FermionOperators can be directly passed in to create a newwavefunction based on application of the operators.

In [42]:
from openfermion import FermionOperator
ops = FermionOperator('2^ 0', 1.2)
new_wfn = wfn_from_cirq.apply(ops)

In [43]:
new_wfn = fqe.apply(FermionOperator('0^ 0', 1.0), wfFQE)
if fqeprint:
    wfFQE.print_wfn()
    new_wfn.print_wfn()

Sector N = 2 : S_z = -2
a'0000'b'0011' (0.13741376984264198+0.21858138930916632j)
a'0000'b'0101' (0.17812903120313933+0.23050144667372388j)
a'0000'b'1001' (0.018493928708758628-0.803290479161759j)
a'0000'b'0110' (-0.01790010542315535-0.10757020379541009j)
a'0000'b'1010' (0.1077834079098812+0.3317034329962844j)
a'0000'b'1100' (-0.04538753591859354-0.25935482008351124j)
Sector N = 2 : S_z = 0
a'0001'b'0001' (-0.032160217105011144-0.10925842023669652j)
a'0001'b'0010' (-0.09759469091924397-0.08998677749039163j)
a'0001'b'0100' (0.22406346618992173+0.12802183687064878j)
a'0001'b'1000' 3.14159265359j
a'0010'b'0001' (0.1233684413471275-0.16701120663809563j)
a'0010'b'0010' (0.08465086261327827-0.09816117743994539j)
a'0010'b'0100' (0.20059792781307756-0.22999939274175885j)
a'0010'b'1000' (0.07310730363588082-0.12480494955030182j)
a'0100'b'0001' (0.06651284373056578-0.4142631154525293j)
a'0100'b'0010' (0.31429653318969253-0.1030206703270467j)
a'0100'b'0100' (0.16285108221656122-0.0770395062669499

The fqe has vdot functionality to get an inner product and provides a way to do completely general expectation values.

In [44]:
ops = FermionOperator('2^ 2', 0.5)
opexpec = fqe.vdot(wfFQE, wfFQE.apply(ops))
if fqeprint:
    print(opexpec)

(0.1908362624309661+0j)


In [45]:
ops = FermionOperator('2^ 2', 0.5)
opexpec = fqe.vdot(wfFQE, fqe.apply(ops, wfFQE))
if fqeprint:
    print(opexpec)

(0.1908362624309661+0j)


We can also do general unitary application of operators.  One slight modification to the proposal is that the operator passed in should Hermetian and not anti-Hermitian as stated before.

In [46]:
ops = FermionOperator('2^ 0', 0.2 - .3j) + FermionOperator('0^ 2', 0.2 + .3j)
wfFQE.print_wfn()
wfFQE2 = wfFQE.apply_generated_unitary(0.001, 'taylor', ops)
wfFQE2.print_wfn()
ops = FermionOperator('2^ 0', 0.2 - .3j) + FermionOperator('0^ 2', 0.2 + .3j)
wfFQE2 = fqe.apply_generated_unitary(wfFQE, 0.001, 'taylor', ops)
wfFQE2.print_wfn()

Sector N = 2 : S_z = -2
a'0000'b'0011' (0.13741376984264198+0.21858138930916632j)
a'0000'b'0101' (0.17812903120313933+0.23050144667372388j)
a'0000'b'1001' (0.018493928708758628-0.803290479161759j)
a'0000'b'0110' (-0.01790010542315535-0.10757020379541009j)
a'0000'b'1010' (0.1077834079098812+0.3317034329962844j)
a'0000'b'1100' (-0.04538753591859354-0.25935482008351124j)
Sector N = 2 : S_z = 0
a'0001'b'0001' (-0.032160217105011144-0.10925842023669652j)
a'0001'b'0010' (-0.09759469091924397-0.08998677749039163j)
a'0001'b'0100' (0.22406346618992173+0.12802183687064878j)
a'0001'b'1000' 3.14159265359j
a'0010'b'0001' (0.1233684413471275-0.16701120663809563j)
a'0010'b'0010' (0.08465086261327827-0.09816117743994539j)
a'0010'b'0100' (0.20059792781307756-0.22999939274175885j)
a'0010'b'1000' (0.07310730363588082-0.12480494955030182j)
a'0100'b'0001' (0.06651284373056578-0.4142631154525293j)
a'0100'b'0010' (0.31429653318969253-0.1030206703270467j)
a'0100'b'0100' (0.16285108221656122-0.0770395062669499

### 3.B.1 Hamiltonian data structure

Hamiltonians can be incorporated from OpenFermion objects or initialized from numpy arrays.

In [47]:
from openfermion import get_quadratic_hamiltonian
ops = FermionOperator((), 1.) + FermionOperator('1^ 1', 3.) + FermionOperator('1^ 2', 3. + 4.j) + FermionOperator('2^ 1', 3. - 4.j) + FermionOperator('3^ 4', 2. + 5.j) + FermionOperator('4^ 3', 2. - 5.j)
#hamOF = get_quadratic_hamiltonian(ops, -0.03)
hamFQE = fqe.get_hamiltonian_from_openfermion(ops, e_0=-0.03)
#hamFQE = fqe.get_quadratic_hamiltonian(ops, -0.03)

Symmetry is passed in by lists containing permutation operations, a phase and whether or not complex conjugation is used.  Below are examples of a Hermetian matrix and a real anti-symmetric matrix.

In [48]:
if False:
    e = 1.0
    t = numpy.random.rand(4, 4) + numpy.random.rand(4, 4)*1.j
    h = t + numpy.conjugate(t.T)
    symmh = [
        [[1, 2], 1.0, False],
        [[2, 1], 1.0, True]
    ]
    t = numpy.random.rand(4, 4, 4, 4)
    g = t - t.T
    symmg = [
        [[1, 2, 3, 4], 1.0, True],
        [[4, 3, 2, 1], -1.0, True]
    ]
    hamFQE = fqe.get_two_body_hamiltonian(-0.3, h, g, 0.0, symmh, symmg)

The FQE will also return PolynomialTensor objects for parsing by OpenFermion objects.

In [49]:
#hamOF = fqe.hamiltonian_to_openfermion(hamFQE)

### Saving and Loading Wavefunctions

Saving or loading a wavefunction is accomplished by passing a file name to the API and optionally a path to that filename.  If no path is given, the current working directory will be used.  If a path is used, the file will be searched for as

path + '/' + filename

In [50]:
wfnFqe = fqe.Wavefunction([[2, 2, 2], [2, 0, 2], [2, -2, 2]], broken=None)
wfnFqe.set_wfn(strategy='random')
wfnFqe.print_wfn()

Sector N = 2 : S_z = -2
a'00'b'11' (-0.6482451936912013+0.7614316573772442j)
Sector N = 2 : S_z = 0
a'01'b'01' (0.04597989529030235-0.12788798123539336j)
a'01'b'10' (0.41463911740062676+0.16377546743839544j)
a'10'b'01' (-0.4422799709909502+0.3172576946978844j)
a'10'b'10' (0.6637737459222839+0.21429631053921616j)
Sector N = 2 : S_z = 2
a'11'b'00' (-0.07442437560674867+0.9972266604516478j)


In [51]:
wfnFqe.save('random_wavefunction')
# wfnFqe.save('random_wavefunction', path='/home/google/wavefunctions')

Once it is saved you can return to the wavefunction by creating new object and then 'read' the data into it with the same syntax.

In [52]:
progress = fqe.Wavefunction([], broken=None)
progress.read('random_wavefunction')
# progress.read('random_wavefunction', path='/home/google/wavefunctions')
progress.print_wfn()

Sector N = 2 : S_z = -2
a'00'b'11' (-0.6482451936912013+0.7614316573772442j)
Sector N = 2 : S_z = 0
a'01'b'01' (0.04597989529030235-0.12788798123539336j)
a'01'b'10' (0.41463911740062676+0.16377546743839544j)
a'10'b'01' (-0.4422799709909502+0.3172576946978844j)
a'10'b'10' (0.6637737459222839+0.21429631053921616j)
Sector N = 2 : S_z = 2
a'11'b'00' (-0.07442437560674867+0.9972266604516478j)
